In [334]:
from selenium import webdriver
import selenium
import time
from bs4 import BeautifulSoup
import os
from selenium.webdriver.common.keys import Keys
import math
import pandas as pd
import random
from selenium.webdriver.support.ui import Select

In [335]:
#1. 사용자 인터페이스 만들기 
print('='*100)
print('네이버 쇼핑 상품 크롤러[가격]')
cnt = int(input('총 몇 건 상품정보를 크롤링 하시겠습니까? '))
key_word = input('검색할 키워드를 입력하세요(예: 귀여운다꾸스티커) : ')
f = input('파일을 저장할 경로를 입력하세요')
if f == "" : 
    f = '/Users/kibeomkim/Desktop/웹크롤링_결과_데이터/'
print('='*100)


now = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' %(now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)

os.makedirs(f+s+'-'+'네이버 쇼핑 상품 정보')
os.chdir(f+s+'-'+'네이버 쇼핑 상품 정보')

f_name = s+'-'+'네이버 쇼핑 상품 정보'+'.txt'
fc_name = s+'-'+'네이버 쇼핑 상품 정보'+'.csv'
fx_name = s+'-'+'네이버 쇼핑 상품 정보'+'.xlsx'


네이버 쇼핑 상품 크롤러[가격]
총 몇 건 상품정보를 크롤링 하시겠습니까? 800
검색할 키워드를 입력하세요(예: 귀여운다꾸스티커) : 
파일을 저장할 경로를 입력하세요


In [336]:
#2. 웹드라이버 호출
path = '/Users/kibeomkim/Desktop/chromedriver_240/chromedriver'
options = webdriver.ChromeOptions()
options.add_argument('User-Agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 11_1_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36')

driver = webdriver.Chrome(path, options=options)

s_time = time.time()

In [337]:
#3. 네이버 쇼핑 페이지로 이동
driver.get('https://www.google.com/search?hl=ko&tbm=shop&source=hp&ei=mi8MYObcEKvVmAWB0ragBA&q=%EA%B7%80%EC%97%AC%EC%9A%B4+%EB%8B%A4%EA%BE%B8+%EC%8A%A4%ED%8B%B0%EC%BB%A4&oq=%EA%B7%80%EC%97%AC%EC%9A%B4+%EB%8B%A4%EA%BE%B8+%EC%8A%A4%ED%8B%B0%EC%BB%A4&gs_lcp=Cgtwcm9kdWN0cy1jYxADOgIIAFCTCFj8JmDpJ2gGcAB4BYABlAGIAdcUkgEEMC4yMJgBAKABAaoBD3Byb2R1Y3RzLWNjLXdpeg&sclient=products-cc&ved=0ahUKEwjmke-Kn7LuAhWrKqYKHQGpDUQQ4dUDCAc&uact=5')
time.sleep(3)

driver.find_element_by_class_name('vkYnff').click()
time.sleep(3)
driver.find_element_by_xpath('/html/body/div[7]/div[2]/div[5]/div/g-menu/g-menu-item[2]/div').click()
time.sleep(3)

f = open(f_name, 'a', encoding='utf-8')

In [338]:
name_list = []
price_list = []
no1 = 1
no2 = 2
no3 = 1

while no2 == 2 : 
    html2 = driver.page_source
    soup2 = BeautifulSoup(html2, 'html.parser')
    list2 = soup2.find('div', 'sh-pr__product-results-grid sh-pr__product-results').find_all('div', 'sh-dgr__gr-auto sh-dgr__grid-result')
    for a in list2 : 
        name = a.find('h4', 'A2sOrd').get_text().replace('\n', "").strip()
        f.write('상품 이름 : '+name +'\n')
        name_list.append(name)


    
        price = a.find('span', 'QIrs8').get_text().replace('₩', "").strip()
        f.write('가격 : '+price+'\n')
        price_list.append(price)


    
        no1 += 1 
        if no1 > cnt : 
            break

    no3 += 1 
    try : 
        driver.find_element_by_xpath('/html/body/div[7]/div[2]/div[8]/div[2]/div/div[5]/div[2]/span[1]/div[1]/table/tbody/tr/td[%s]/a' %(no3+1)).click()
        time.sleep(3)
    except : 
        break
        
f.close()
driver.close()

price2 = []
for c in price_list : 
    d = c.replace(',', "").replace(".", "").strip()
    price2.append(d)
    
cont = pd.DataFrame()
cont['제목'] = name_list
cont['가격'] = price2

cont.to_csv(fc_name, encoding='utf-8-sig', index=False)
cont.to_excel(fx_name)



In [339]:
df = pd.read_csv('/Users/kibeomkim/Desktop/afa.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 731 entries, 0 to 730
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   제목      731 non-null    object
 1   가격      731 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 11.5+ KB


In [340]:
df1 = df[pd.notnull(df['제목'])]
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 731 entries, 0 to 730
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   제목      731 non-null    object
 1   가격      731 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 17.1+ KB


In [341]:
df1.describe()

가격
count    731.000000
mean    6485.300958
std     6999.657222
min      700.000000
25%     2000.000000
50%     4150.000000
75%     8635.000000
max    65900.000000